# Notebook to demonstrate Zero shot and Few shot Learning

In [1]:
import pandas as pd 
#%pip install langchain_groq
from langchain_groq.chat_models import ChatGroq
import pandas as pd
import numpy as np
from itertools import product
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score

In [2]:
# Groq API and Models 
Groq_Token = open('..\\groqapi.txt', 'r').read()  # Do not share this key with anyone
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

**NOTE : DO NOT SHARE THE API KEY WITH ANYONE. DO NOT COMMIT THE API KEY TO GITHUB.**

Always do a sanity check before committing the code to github. If the key is found in the code, you will be penalized with a 0.5 marks deduction.

# Zero Shot 

In [114]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."

# System Prompts 
query = f"""
* You are a sentiment analysis model. 
* Your task is to analyze the sentiment expressed in the given text and classify it as 'positive', 'negative', or 'neutral'. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.

Sentence: {sentence}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Sentiment label: Neutral

Explanation: The sentence expresses mixed sentiments. The words "amazing" and "happy" convey a positive sentiment, indicating satisfaction with the product quality and customer service. However, the phrase "delivery was delayed" expresses a negative sentiment, indicating dissatisfaction with the delivery experience. Overall, the positive and negative sentiments balance each other out, resulting in a neutral sentiment label.


# Few Shot

In [4]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."

# System Prompts 
query = f"""
* You are a sentiment analysis model. 
* Your task is to analyze the sentiment expressed in the given text and classify it as 'positive', 'negative', or 'neutral'. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.

Here are few examples:
1. Sentence: 'The customer service was excellent, and I received my order quickly.'
Sentiment: Positive

2. Sentence: 'The food was bland and the service was slow.'
Sentiment: Negative

3. Sentence: 'The product is okay, but it's not worth the price.'
Sentiment: Neutral

Sentence: {sentence}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Sentiment: Positive

Explanation: Although the sentence mentions a negative aspect ("the delivery was delayed"), the positive sentiments ("The product quality is amazing" and "I am happy with the customer service") outweigh the negative one, resulting in an overall positive sentiment. The use of the word "amazing" and "happy" also indicates a strong positive emotion, which contributes to the positive sentiment classification.


In [3]:
f = open("C:\\Users\\Rajeev Wankar\\Desktop\\Aarsh IITGN\\ES335 Machine Learning\\main\\ES335-MLAssignment1\\HAR\\UCI HAR Dataset\\features.txt", 'r').read().split('\n')

In [4]:
feature_labels = pd.read_csv(r'../HAR/UCI HAR Dataset/features.txt',sep = '\\s+',header=None)
dataframeX = pd.read_csv(r'../HAR/UCI HAR Dataset/train/X_train.txt',sep = '\\s+',header=None)
dataframeY = pd.read_csv(r'../HAR/UCI HAR Dataset/train/y_train.txt',sep = '\\s+',header=None)
correlation_matrix = dataframeX.corr()

In [5]:
def reduce_features(threshold = None):
    #feature_labels = pd.read_csv(r'../HAR/UCI HAR Dataset/features.txt',sep = '\\s+',header=None)
    #dataframeX = pd.read_csv(r'../HAR/UCI HAR Dataset/train/X_train.txt',sep = '\\s+',header=None)
    #dataframeY = pd.read_csv(r'../HAR/UCI HAR Dataset/train/y_train.txt',sep = '\\s+',header=None)
    #test_values_y = pd.read_csv(r'../HAR/UCI HAR Dataset/test/y_test.txt',sep = "\\s+",header=None)
    #test_data_X = pd.read_csv(r'../HAR/UCI HAR Dataset/test/X_test.txt',sep = "\\s+",header=None)
    #correlation_matrix = dataframeX.corr()
    removals = set()
    pairs = product(range(len(correlation_matrix.columns)), repeat=2)

    for i, j in pairs:
        if i > j:
            correlation_value = correlation_matrix.iloc[i, j]
            if abs(correlation_value) > threshold and correlation_value != 1:
                if i not in removals and j not in removals:
                    removals.add(i)
    #print(removals)
    selected_features = [col for col in dataframeX.columns if col not in removals]
    return selected_features
    
    # clf = tree.DecisionTreeClassifier(max_depth = depth)
    # clf = clf.fit(filtered_dfX, dataframeY)
    # tree.plot_tree(clf)
    
    # filtered_testx =test_data_X[selected_features]
    
    # y_pred = clf.predict(filtered_testx)
    # y_pred = np.array(y_pred)
    # y_test = test_values_y[0].to_numpy()
    # print(filtered_testx.shape)
    # accuracy = accuracy_score(y_test, y_pred)
    # print(f"Accuracy: {accuracy}")
    # precision = precision_score(y_test, y_pred,average='weighted')
    # print(f"Precision: {precision}")
    # recall = recall_score(y_test, y_pred, average='weighted')
    # print(f"Recall: {recall}")
    # cm = confusion_matrix(y_test, y_pred)
    # print("Confusion Matrix:")
    # print(cm)

    

In [6]:
frs = reduce_features(threshold=0.9)

In [7]:
features = np.array(feature_labels[1].iloc[frs])

def example(n):
    return dataframeX[frs].iloc[n, :].to_numpy()

activity_dic = {1: "WALKING", 2: "WALKING_UPSTAIRS", 3: "WALKING_DOWNSTAIRS", 4: "SITTING", 5: "STANDING", 6: "LAYING"}
def label(n):
    #print(dataframeY.iloc[n, 0])
    return (activity_dic[dataframeY.iloc[n, 0]])
    
label(10)
print(label(100))
print(label(234))
print(label(134))
print(label(135))

WALKING
SITTING
WALKING_DOWNSTAIRS
WALKING_DOWNSTAIRS


In [143]:
features

array(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Z', 'tBodyAcc-max()-Y',
       'tBodyAcc-energy()-Y', 'tBodyAcc-entropy()-X',
       'tBodyAcc-entropy()-Y', 'tBodyAcc-entropy()-Z',
       'tBodyAcc-arCoeff()-X,1', 'tBodyAcc-arCoeff()-X,2',
       'tBodyAcc-arCoeff()-X,3', 'tBodyAcc-arCoeff()-X,4',
       'tBodyAcc-arCoeff()-Y,1', 'tBodyAcc-arCoeff()-Y,2',
       'tBodyAcc-arCoeff()-Y,3', 'tBodyAcc-arCoeff()-Y,4',
       'tBodyAcc-arCoeff()-Z,1', 'tBodyAcc-arCoeff()-Z,2',
       'tBodyAcc-arCoeff()-Z,3', 'tBodyAcc-arCoeff()-Z,4',
       'tBodyAcc-correlation()-X,Y', 'tBodyAcc-correlation()-X,Z',
       'tBodyAcc-correlation()-Y,Z', 'tGravityAcc-mean()-X',
       'tGravityAcc-mean()-Y', 'tGravityAcc-mean()-Z',
       'tGravityAcc-std()-X', 'tGravityAcc-std()-Y',
       'tGravityAcc-std()-Z', 'tGravityAcc-sma()',
       'tGravityAcc-energy()-Y', 'tGravityAcc-energy()-Z',
       'tGravityAcc-entropy()-X', 'tGravityAcc-entropy

In [33]:
dictionary = {}
for i in range(len(frs)):
    dictionary[frs[i]] = features[i]

In [34]:
dictionary

{0: 'tBodyAcc-mean()-X',
 1: 'tBodyAcc-mean()-Y',
 2: 'tBodyAcc-mean()-Z',
 3: 'tBodyAcc-std()-X',
 5: 'tBodyAcc-std()-Z',
 10: 'tBodyAcc-max()-Y',
 17: 'tBodyAcc-energy()-Y',
 22: 'tBodyAcc-entropy()-X',
 23: 'tBodyAcc-entropy()-Y',
 24: 'tBodyAcc-entropy()-Z',
 25: 'tBodyAcc-arCoeff()-X,1',
 26: 'tBodyAcc-arCoeff()-X,2',
 27: 'tBodyAcc-arCoeff()-X,3',
 28: 'tBodyAcc-arCoeff()-X,4',
 29: 'tBodyAcc-arCoeff()-Y,1',
 30: 'tBodyAcc-arCoeff()-Y,2',
 31: 'tBodyAcc-arCoeff()-Y,3',
 32: 'tBodyAcc-arCoeff()-Y,4',
 33: 'tBodyAcc-arCoeff()-Z,1',
 34: 'tBodyAcc-arCoeff()-Z,2',
 35: 'tBodyAcc-arCoeff()-Z,3',
 36: 'tBodyAcc-arCoeff()-Z,4',
 37: 'tBodyAcc-correlation()-X,Y',
 38: 'tBodyAcc-correlation()-X,Z',
 39: 'tBodyAcc-correlation()-Y,Z',
 40: 'tGravityAcc-mean()-X',
 41: 'tGravityAcc-mean()-Y',
 42: 'tGravityAcc-mean()-Z',
 43: 'tGravityAcc-std()-X',
 44: 'tGravityAcc-std()-Y',
 45: 'tGravityAcc-std()-Z',
 55: 'tGravityAcc-sma()',
 57: 'tGravityAcc-energy()-Y',
 58: 'tGravityAcc-energy()-Z',
 

In [45]:
act_labels = [label(i) for i in range(7352)]

In [48]:
reduced_dataset = dataframeX[frs]
reduced_dataset=reduced_dataset.rename(columns=dictionary)

#display(reduced_dataset)
reduced_dataset.insert(0, "Activity_Name", act_labels)
#reduced_dataset["Activity_Name"] = act_labels
display(reduced_dataset)

,Activity_Name,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Z,tBodyAcc-max()-Y,tBodyAcc-energy()-Y,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,...,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)"
0,STANDING,0.288585,-0.020294,-0.132905,-0.995279,-0.913526,-0.567378,-0.999863,-0.407747,-0.679338,...,-0.128989,0.586156,-0.991048,-1.000000,-0.074323,-0.298676,-0.112754,0.030400,-0.464761,-0.018446
1,STANDING,0.278419,-0.016411,-0.123520,-0.998245,-0.960322,-0.557851,-0.999788,-0.714892,-0.500930,...,-0.271958,-0.336310,-0.994440,-1.000000,0.158075,-0.595051,0.053477,-0.007435,-0.732626,0.703511
2,STANDING,0.279653,-0.019467,-0.113462,-0.995380,-0.978944,-0.557851,-0.999660,-0.592235,-0.485821,...,-0.212728,-0.535352,-0.995866,-0.555556,0.414503,-0.390748,-0.118559,0.177899,0.100699,0.808529
3,STANDING,0.279174,-0.026201,-0.123283,-0.996091,-0.990675,-0.576159,-0.999736,-0.627446,-0.850930,...,-0.035684,-0.230091,-0.995732,-0.936508,0.404573,-0.117290,-0.036788,-0.012892,0.640011,-0.485366
4,STANDING,0.276629,-0.016570,-0.115362,-0.998139,-0.990482,-0.569174,-0.999856,-0.786553,-0.559477,...,-0.273582,-0.510282,-0.997418,-0.936508,0.087753,-0.351471,0.123320,0.122542,0.693578,-0.615971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,WALKING_UPSTAIRS,0.299665,-0.057193,-0.181233,-0.195387,0.077078,0.029369,-0.788077,0.195643,0.140158,...,-0.376234,-0.170845,-0.925241,-0.904762,-0.070157,-0.588433,-0.190437,0.829718,0.206972,-0.425619
7348,WALKING_UPSTAIRS,0.273853,-0.007749,-0.147468,-0.235309,0.059280,0.029369,-0.804546,0.178138,0.293566,...,-0.296176,0.019626,-0.761880,-0.904762,0.165259,-0.390738,0.064907,0.875679,-0.879033,0.400219
7349,WALKING_UPSTAIRS,0.273387,-0.017011,-0.045022,-0.218218,0.274533,-0.110405,-0.844279,0.173131,0.276040,...,-0.320249,0.490169,-0.692582,-0.904762,0.195034,0.025145,0.052806,-0.266724,0.864404,0.701169
7350,WALKING_UPSTAIRS,0.289654,-0.018843,-0.158281,-0.219139,0.268893,-0.149495,-0.846871,0.160333,0.246561,...,-0.412332,0.308105,-0.886154,-0.904762,0.013865,0.063907,-0.101360,0.700740,0.936674,-0.589479


In [61]:
reduced_dataset

,Activity_Name,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Z,tBodyAcc-max()-Y,tBodyAcc-energy()-Y,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,...,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)"
0,STANDING,0.288585,-0.020294,-0.132905,-0.995279,-0.913526,-0.567378,-0.999863,-0.407747,-0.679338,...,-0.128989,0.586156,-0.991048,-1.000000,-0.074323,-0.298676,-0.112754,0.030400,-0.464761,-0.018446
1,STANDING,0.278419,-0.016411,-0.123520,-0.998245,-0.960322,-0.557851,-0.999788,-0.714892,-0.500930,...,-0.271958,-0.336310,-0.994440,-1.000000,0.158075,-0.595051,0.053477,-0.007435,-0.732626,0.703511
2,STANDING,0.279653,-0.019467,-0.113462,-0.995380,-0.978944,-0.557851,-0.999660,-0.592235,-0.485821,...,-0.212728,-0.535352,-0.995866,-0.555556,0.414503,-0.390748,-0.118559,0.177899,0.100699,0.808529
3,STANDING,0.279174,-0.026201,-0.123283,-0.996091,-0.990675,-0.576159,-0.999736,-0.627446,-0.850930,...,-0.035684,-0.230091,-0.995732,-0.936508,0.404573,-0.117290,-0.036788,-0.012892,0.640011,-0.485366
4,STANDING,0.276629,-0.016570,-0.115362,-0.998139,-0.990482,-0.569174,-0.999856,-0.786553,-0.559477,...,-0.273582,-0.510282,-0.997418,-0.936508,0.087753,-0.351471,0.123320,0.122542,0.693578,-0.615971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,WALKING_UPSTAIRS,0.299665,-0.057193,-0.181233,-0.195387,0.077078,0.029369,-0.788077,0.195643,0.140158,...,-0.376234,-0.170845,-0.925241,-0.904762,-0.070157,-0.588433,-0.190437,0.829718,0.206972,-0.425619
7348,WALKING_UPSTAIRS,0.273853,-0.007749,-0.147468,-0.235309,0.059280,0.029369,-0.804546,0.178138,0.293566,...,-0.296176,0.019626,-0.761880,-0.904762,0.165259,-0.390738,0.064907,0.875679,-0.879033,0.400219
7349,WALKING_UPSTAIRS,0.273387,-0.017011,-0.045022,-0.218218,0.274533,-0.110405,-0.844279,0.173131,0.276040,...,-0.320249,0.490169,-0.692582,-0.904762,0.195034,0.025145,0.052806,-0.266724,0.864404,0.701169
7350,WALKING_UPSTAIRS,0.289654,-0.018843,-0.158281,-0.219139,0.268893,-0.149495,-0.846871,0.160333,0.246561,...,-0.412332,0.308105,-0.886154,-0.904762,0.013865,0.063907,-0.101360,0.700740,0.936674,-0.589479


In [137]:
np.random.seed(0)
total_arr = np.random.choice(7352, 1990, replace=False)
train_arr = total_arr[:1990]
test_arr = total_arr[1990:]

In [138]:
train_dataset = reduced_dataset.loc[train_arr]
test_dataset = reduced_dataset.loc[test_arr].drop(columns="Activity_Name")
display(train_dataset)

,Activity_Name,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Z,tBodyAcc-max()-Y,tBodyAcc-energy()-Y,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,...,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)"
3157,LAYING,0.280312,-0.018644,-0.110949,-0.979961,-0.996597,-0.570323,-0.999804,-0.258567,-0.569629,...,-0.064465,-0.346629,-0.998093,-0.841270,0.194253,-0.599251,0.190918,0.050029,-0.071826,-0.314750
5687,STANDING,0.275667,-0.024431,-0.103512,-0.996606,-0.951425,-0.520545,-0.998852,-0.714892,-0.521302,...,-0.318453,-0.092067,-0.993373,-1.000000,-0.490287,0.008181,-0.012397,-0.142878,0.479600,-0.739328
4140,WALKING_DOWNSTAIRS,0.416676,-0.059575,-0.158889,0.274506,0.047251,0.176037,-0.554872,0.320805,0.360911,...,0.054036,-0.204463,-0.945711,-0.904762,0.267174,0.002234,-0.755331,0.690159,0.876933,-0.631443
4401,STANDING,0.285361,-0.000836,-0.098353,-0.994398,-0.973610,-0.519487,-0.998599,-0.442739,-0.030803,...,-0.715790,0.283664,-0.992680,-1.000000,-0.469025,0.106802,0.001516,0.122648,-0.247219,-0.594063
6518,LAYING,0.262276,-0.037320,-0.101357,-0.986755,-0.976628,-0.513497,-0.997402,-0.760077,-0.506618,...,-0.162630,-0.062568,-0.977692,-1.000000,-0.134382,-0.231558,0.014016,0.279879,-0.229328,0.225381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,SITTING,0.281468,-0.017691,-0.111765,-0.997293,-0.996573,-0.570965,-0.999970,-0.596409,-0.726470,...,0.216148,-0.302892,-0.997888,-1.000000,0.327995,-0.281085,0.064921,0.000456,0.064799,0.023543
4145,WALKING_UPSTAIRS,0.314266,-0.054812,-0.143089,-0.278212,-0.307999,-0.007187,-0.726461,0.449300,0.146960,...,-0.297487,-0.164442,-0.673972,-0.904762,0.340255,-0.227751,-0.543125,0.528985,-0.959434,-0.214053
3892,LAYING,0.269315,-0.017069,-0.108901,-0.955115,-0.985636,-0.541344,-0.999794,-0.112606,-0.580046,...,0.760829,-0.551067,-0.977897,-0.968254,-0.687828,-0.253989,-0.120043,0.617006,-0.336063,-0.019954
7168,STANDING,0.282962,-0.056689,-0.165917,-0.970968,-0.831214,-0.496127,-0.983831,-0.234426,-0.318050,...,-0.194493,-0.291106,-0.972220,-1.000000,0.406974,-0.043132,0.010188,0.452593,-0.224437,-0.139538


In [139]:
display(test_dataset)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Z,tBodyAcc-max()-Y,tBodyAcc-energy()-Y,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,...,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)"
1533,0.279761,-0.017036,-0.109688,-0.964245,-0.989495,-0.564397,-0.999756,-0.187142,-0.489921,-0.611571,...,-0.494896,0.389621,-0.989677,-0.936508,0.180544,-0.223170,0.430387,0.033084,-0.447991,0.570442
2404,0.284711,-0.013662,-0.099579,-0.940141,-0.886519,-0.482525,-0.995784,0.085881,-0.046080,-0.110825,...,-0.086160,-0.396150,-0.956196,-0.269841,-0.180443,-0.492014,0.045816,0.001683,-0.431990,0.214116
5532,0.289049,-0.016569,-0.118212,-0.992741,-0.990523,-0.570201,-0.999970,-0.366178,-0.698294,-0.798709,...,-0.121271,-0.532933,-0.992840,-0.936508,0.238137,-0.743669,0.031945,0.204304,0.157495,-0.173260
2819,0.297488,0.009518,-0.118657,-0.300347,-0.629125,0.053406,-0.717715,0.462755,0.419738,0.022050,...,0.175699,-0.581824,-0.699010,-0.873016,0.033397,0.009262,-0.039846,-0.293072,-0.709643,0.150273
1732,0.227831,-0.028491,-0.067252,0.025241,0.196887,0.283700,-0.568415,0.195643,0.234327,0.347650,...,0.288588,-0.601432,-0.828003,-0.873016,0.225144,-0.243161,0.348493,-0.462526,0.780309,-0.738280
2459,0.257612,-0.022177,-0.102217,-0.968543,-0.942193,-0.528341,-0.999019,-0.444423,-0.497341,-0.219617,...,0.404440,-0.565639,-0.948043,-1.000000,0.051921,-0.262105,0.112025,-0.400353,-0.077662,0.173592
113,0.292724,-0.002140,-0.140349,-0.252537,-0.275239,-0.113901,-0.789735,0.345317,0.441412,-0.092874,...,0.305295,0.226689,-0.787206,-0.873016,0.248022,-0.047307,-0.065761,-0.765500,0.810453,0.090855
3864,0.146257,-0.029772,-0.114904,-0.904819,-0.978289,-0.552591,-0.999205,-0.382164,-0.734129,-0.573565,...,-0.317978,-0.321327,-0.977564,-1.000000,-0.092584,-0.022904,-0.034099,0.179870,-0.105370,0.744279
6927,0.139274,0.029781,-0.038040,0.229213,-0.307351,0.145525,-0.894040,0.098014,0.337400,0.414475,...,-0.015677,-0.255632,-0.896476,-0.873016,0.034991,-0.277651,0.464842,0.913339,-0.902415,0.426014
4170,0.330572,-0.032711,-0.120317,-0.318413,-0.209273,-0.031693,-0.782250,0.557036,0.153421,0.212032,...,0.107087,-0.169915,-0.943260,-1.000000,0.263478,-0.310298,-0.876120,-0.346397,-0.888205,-0.381105


In [142]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."
# System Prompts 
query = f"""
* You are a Human Activity Recognition model.
* Your task is to analyze the data given in the features provided and guess the activity being performed by the person.
* The activities are: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
* Provide only the activity label as a single word for each sample.

Train data: {train_dataset}

Now, predict the activity label of these examples:
Test data: {test_dataset}
""" 

#To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0.00000001)
answer = llm.invoke(query)

print(answer.content)

Based on the provided test data, I predict the activity labels as follows:

1. STANDING
2. STANDING
3. WALKING
4. WALKING_UPSTAIRS
5. SITTING
6. STANDING
7. STANDING
8. WALKING_DOWNSTAIRS
9. WALKING
10. WALKING_DOWNSTAIRS


In [141]:
for i in range(len(test_arr)):
    print(i+1, label(test_arr[i]))

1 LAYING
2 STANDING
3 SITTING
4 WALKING
5 WALKING_DOWNSTAIRS
6 LAYING
7 WALKING
8 LAYING
9 WALKING_DOWNSTAIRS
10 WALKING_UPSTAIRS


In [56]:
label(135)

'WALKING_DOWNSTAIRS'

In [8]:
import os
from groq import Groq

In [47]:
client = Groq(
    api_key=Groq_Token,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": f"""
                * You are a Human Activity Recognition model.
                * Your task is to analyze the data given in the features provided and guess the activity being performed by the person.
                * The activities are: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
                * The features are: {features}
                * Provide the activity label and, if necessary, a brief explanation of your reasoning.
            """,
        },
        {
            "role": "user",
            "content": f"""
                        Now, predict this:
                        Feature Data: {" ".join(map(str, example(706)))}
            """
        }
    ],
    model=groq_models["llama3-70b"],
    temperature=0
)

print(chat_completion.choices[0].message.content)

Based on the provided feature data, I predict that the activity being performed is **WALKING**.

My reasoning is based on the following observations:

1. The values of `tBodyAcc-mean()-X`, `tBodyAcc-mean()-Y`, and `tBodyAcc-mean()-Z` are relatively small, indicating that the body acceleration is not extreme in any direction. This is consistent with walking, which typically involves gentle movements.
2. The values of `tBodyAcc-std()-X`, `tBodyAcc-std()-Y`, and `tBodyAcc-std()-Z` are moderate, suggesting that there is some variation in the body acceleration, but not excessively so. This is also consistent with walking, which involves some degree of movement variability.
3. The values of `tBodyGyro-mean()-X`, `tBodyGyro-mean()-Y`, and `tBodyGyro-mean()-Z` are relatively small, indicating that the body angular velocity is not extreme in any direction. This is consistent with walking, which typically involves gentle rotations.
4. The values of `fBodyAcc-bandsEnergy()-1,8`, `fBodyAcc-bandsEn

In [10]:
client = Groq(
    api_key=Groq_Token,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": f"""
                * You are a Human Activity Recognition model.
                * Your task is to analyze the data given in the features provided and guess the activity being performed by the person.
                * The activities are: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
                * The features are: {features}
                * Provide the activity label and, if necessary, a brief explanation of your reasoning.
            """,
        },
        {
            "role": "user",
            "content": f"""
                        Look at these examples for reference:
                        Example 1: 
                        
                        Feature Data: {" ".join(map(str, example()))}
                        Activity Label: {label(489)}

                        Example 2:
                        Feature Data: {" ".join(map(str, example(100)))}
                        Activity Label: {label(100)}

                        Example 3:
                        Feature Data: {" ".join(map(str, example(345)))}
                        Activity Label: {label(345)}
                        
                        Now, predict this:
                        Feature Data: {" ".join(map(str, example(706)))}
            """
        }
    ],
    #model=groq_models["llama3-70b"],
    model = "llama-3.1-70b-versatile",
    temperature=0
)

print(chat_completion.choices[0].message.content)

Based on the provided feature data, I predict the activity label to be WALKING_DOWNSTAIRS.

Here's a brief explanation of my reasoning:

1. The feature data shows a mix of high and low values, indicating a dynamic activity.
2. The values of tBodyAcc-mean()-X, tBodyAcc-mean()-Y, and tBodyAcc-mean()-Z are relatively high, suggesting a significant amount of acceleration in the X, Y, and Z axes.
3. The values of tBodyAcc-std()-X, tBodyAcc-std()-Y, and tBodyAcc-std()-Z are relatively high, indicating a high standard deviation of acceleration in the X, Y, and Z axes.
4. The values of tBodyGyro-mean()-X, tBodyGyro-mean()-Y, and tBodyGyro-mean()-Z are relatively low, suggesting a relatively low angular velocity in the X, Y, and Z axes.
5. The values of fBodyAcc-bandsEnergy()-1,8, fBodyAcc-bandsEnergy()-9,16, and fBodyAcc-bandsEnergy()-17,24 are relatively high, indicating a significant amount of energy in the frequency bands corresponding to walking.
6. The values of fBodyAccJerk-bandsEnergy()

In [16]:
act_dic = {"WALKING":[], "WALKING_UPSTAIRS": [], "WALKING_DOWNSTAIRS": [], "SITTING": [], "STANDING": [], "LAYING": []}

In [17]:
for i in range(len(dataframeY[0].to_numpy())):
    act_dic[label(i)].append(i)

In [21]:
act_dic.keys()

dict_keys(['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING'])